# Phosphorylation Density Analysis

## Setup

In [1]:
# Standard Library Imports
from collections import defaultdict
import pathlib

# External Imports
import networkx as nx
import numpy as np
import pandas as pd
import metworkpy

# Local Imports

In [2]:
DATA_PATH = pathlib.Path("..") / "data"
RESULTS_PATH = pathlib.Path("..") / "results"

In [3]:
# List of Kinases
kinase_list = ["PknB", "PknD", "PknE", "PknF", "PknG", "PknH", "PknI", "PknJ", "PknK", "PknL"]

## Read in Gene Info

In [4]:
gene_info_df = pd.read_csv(DATA_PATH / "gene_data" / 
                          "Mycobacterium_tuberculosis_H37Rv_txt_v4.txt",
                          sep="\t").set_index("Locus")

## Read in Phosphorylation Data

In [ ]:
phosphorylations_dict = {}
for kinase in kinase_list:
    df = pd.read_excel(DATA_PATH / "phosphorylation" / "frando_phosphorylation_data.xlsx",
                      sheet_name=kinase,
                      index_col=None, 
                      header=0)
    phosphorylations_dict[kinase] = df.groupby("Rv Number")["Phosphosite"].count()

## Prepare Network

### Read in Model

In [ ]:
model = metworkpy.read_model(str(DATA_PATH / "models" / "iEK1011_m7H10_media.json"))

### Convert Model to Network

In [ ]:
REMOVE_LIST = ["h_c", "h2o_c", "atp_c", "pi_c", "coa_c","adp_c", "co2_c", "nad_c", "ppi_c", "nadh_c", "biomass", "nadp_c", "nadph_c", "amp_c"]

metabolic_network = metworkpy.create_network(model = model, 
                                            weighted = False,
                                            directed=False, 
                                            nodes_to_remove=REMOVE_LIST)

### Project Network onto Reactions

In [ ]:
rxn_list = model.reactions.list_attr("id")
rxn_list.remove("biomass")

reaction_network = metworkpy.bipartite_project(metabolic_network,
                                               node_set = rxn_list,
                                              directed=False)

## Prepare Label Data

In [ ]:
label_dict = {}
for kinase, targets in phosphorylations_dict.items():
    rxn_dict = defaultdict(int)
    for gene, value in targets.items():
        if gene in model.genes:
            for rxn in model.genes.get_by_id(gene).reactions:
                rxn_dict[rxn.id] += value 
    label_dict[kinase] = rxn_dict

## Find Density and Clusters

In [ ]:
RADIUS = 0

rxn_density_series_list = []
for kinase, rxn_dict in label_dict.items():
    density_series = metworkpy.label_density(reaction_network, 
                                            labels = rxn_dict,
                                            radius=RADIUS)
    density_series.name = kinase
    density_series = (density_series - density_series.min())/(density_series.max() - density_series.min())
    rxn_density_series_list.append(density_series)
    density_series.to_json(RESULTS_PATH / "escher" / "rxn_density" /f"{kinase}_rxn_density.json")
    cluster_df = metworkpy.find_dense_clusters(
        network=reaction_network,
        labels=rxn_dict, 
        radius=RADIUS, 
        quantile_cutoff=0.20,
    )
    gene_cluster_df = metworkpy.reaction_to_gene_df(
        model=model, 
        reaction_df = cluster_df,
    )
    gene_cluster_df["density"] = (gene_cluster_df["density"] - gene_cluster_df["density"].min())/(gene_cluster_df["density"].max() - gene_cluster_df["density"].min())
    gene_cluster_df = gene_cluster_df.merge(gene_info_df, 
                         left_index=True,
                         right_index=True, how="left")
    gene_cluster_df.to_csv(RESULTS_PATH / f"{kinase}_density_clusters.csv")
    

rxn_density_df = pd.concat(rxn_density_series_list, axis=1)
rxn_density_df.to_csv(RESULTS_PATH / "reaction_phosphorylation_density.csv")
gene_density_df = metworkpy.reaction_to_gene_df(model=model, reaction_df = rxn_density_df)
gene_density_df = gene_density_df.merge(gene_info_df, left_index=True, 
                     right_index=True, how="left")
gene_density_df.to_csv(RESULTS_PATH / "gene_phosphorylation_density.csv")
